In [19]:
from dotenv import load_dotenv

load_dotenv()


True

## Setup Tools


In [20]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [21]:
from typing import Dict, Any
from tavily import TavilyClient
from langchain.tools import tool

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [22]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

@tool
def query_playlist_db(query: str) -> str:

    """Query the database for playlist information"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"

In [54]:
!sqlite3 resources/Chinook.db ".tables"

Album          Employee       InvoiceLine    PlaylistTrack
Artist         Genre          MediaType      Track        
Customer       Invoice        Playlist     


In [56]:
!sqlite3 resources/Chinook.db "SELECT Track.Name, Genre.Name FROM Track JOIN Genre ON Track.GenreId = Genre.GenreId;"

For Those About To Rock (We Salute You)|Rock
Balls to the Wall|Rock
Fast As a Shark|Rock
Restless and Wild|Rock
Princess of the Dawn|Rock
Put The Finger On You|Rock
Let's Get It Up|Rock
Inject The Venom|Rock
Snowballed|Rock
Evil Walks|Rock
C.O.D.|Rock
Breaking The Rules|Rock
Night Of The Long Knives|Rock
Spellbound|Rock
Go Down|Rock
Dog Eat Dog|Rock
Let There Be Rock|Rock
Bad Boy Boogie|Rock
Problem Child|Rock
Overdose|Rock
Hell Ain't A Bad Place To Be|Rock
Whole Lotta Rosie|Rock
Walk On Water|Rock
Love In An Elevator|Rock
Rag Doll|Rock
What It Takes|Rock
Dude (Looks Like A Lady)|Rock
Janie's Got A Gun|Rock
Cryin'|Rock
Amazing|Rock
Blind Man|Rock
Deuces Are Wild|Rock
The Other Side|Rock
Crazy|Rock
Eat The Rich|Rock
Angel|Rock
Livin' On The Edge|Rock
All I Really Want|Rock
You Oughta Know|Rock
Perfect|Rock
Hand In My Pocket|Rock
Right Through You|Rock
Forgiven|Rock
You Learn|Rock
Head Over Feet|Rock
Mary Jane|Rock
Ironic|Rock
Not The Doctor|Rock
Wake Up|Rock
You Oughta Know (Alternate)|

## Create State

In [23]:
from langchain.agents import AgentState

class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    genre: str
    date: str

## Create Subagents


In [24]:
import os
from langchain.agents import create_agent
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

# Travel agent
travel_agent = create_agent(
    model=model,
    tools=tools,
    system_prompt="""
    You are a travel agent. Search for flights to the desired destination wedding location.
    You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
    - Price (lowest, economy class)
    - Duration (shortest)
    - Date (time of year which you believe is best for a wedding at this location)
    To make things easy, only look for one ticket, one way.
    You may need to make multiple searches to iteratively find the best options.
    You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
    Once you have found the best options, let the user know your shortlist of options.
    """
)

In [25]:
# Venue agent
venue_agent = create_agent(
    model=model,
    tools=[web_search],
    system_prompt="""
    You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
    You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
    - Price (lowest)
    - Capacity (exact match)
    - Reviews (highest)
    You may need to make multiple searches to iteratively find the best options.
    """
)

In [43]:
# Playlist agent
playlist_agent = create_agent(
    model=model,
    tools=[query_playlist_db],
    system_prompt="""
    You are a playlist specialist. Query the sql database and curate the perfect playlist for a wedding given a genre.
    Table available are Track and Genre.
    Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
    If you run into errors when querying the database, try to fix them by making changes to the query.
    Do not come back empty handed, keep trying to query the db until you find a list of songs.
    You may need to make multiple queries to iteratively find the best options.
    """
)

In [44]:
from langchain.messages import HumanMessage
from pprint import pprint

response = await playlist_agent.ainvoke(
    {
        "messages": [HumanMessage(content="Create a plylist of genre pop")],
    }
)

pprint(response)

{'messages': [HumanMessage(content='Create a plylist of genre pop', additional_kwargs={}, response_metadata={}, id='1a003517-5c25-41b0-84d0-c1a8aae8e846'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 169, 'total_tokens': 206, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-Cr6bYmIZdDB7ZpMuAG0a8k1Dhwtmf', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 

In [45]:
print(response['messages'][-1].content)

I have curated a playlist of pop songs for the wedding. Below are the details of the selected tracks:

| Track No. | Genre | Author(s) | Duration (seconds) | Cost ($) |
|-----------|-------|-----------|--------------------|----------|
| 323       | Pop   | Cassiano Costa, Cintia Maviane, J.F., Lucas Costa | 205,479 | 0.99  |
| 324       | Pop   | Augusto Conceição, Chiclete Com Banana            | 198,661 | 0.99  |
| 325       | Pop   | Cal Adan, Paulo Levi                             | 194,194 | 0.99  |
| 326       | Pop   | Paulo Levi, W. Rangel                            | 188,630 | 0.99  |
| 327       | Pop   | Jorge Cardoso, Pierre Onasis                     | 230,791 | 0.99  |
| 328       | Pop   | Fábio Nolasco, Gal Sales, Ivan Brasil            | 206,733 | 0.99  |
| 329       | Pop   | Ricardo Engels, Luca Predabom, Maurício Vieira   | 210,155 | 0.99  |
| 330       | Pop   | Luiz Wanderley, Paulo Levi                       | 190,093 | 0.99  |
| 331       | Pop   | Valverde, Gal

## Main Coordinator


In [46]:
from langchain.tools import ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command

@tool
async def search_flights(runtime: ToolRuntime) -> str:
    """Travel agent searches for flights to the desired destination wedding location."""
    origin = runtime.state["origin"]
    destination = runtime.state["destination"]
    date = runtime.state["date"]
    response = await travel_agent.ainvoke({"messages": [HumanMessage(content=f"Find flights from {origin} to {destination} on {date}")]})
    return response['messages'][-1].content

@tool
def search_venues(runtime: ToolRuntime) -> str:
    """Venue agent chooses the best venue for the given location and capacity."""
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    query = f"Find wedding venues in {destination} for {capacity} guests"
    response = venue_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def suggest_playlist(runtime: ToolRuntime) -> str:
    """Playlist agent curates the perfect playlist for the given genre."""
    genre = runtime.state["genre"]
    query = f"Find {genre} tracks for wedding playlist"
    response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, genre: str, date: str, runtime: ToolRuntime) -> str:
    """Update the state when you know all of the values: origin, destination, guest_count, genre, date"""
    return Command(update={
        "origin": origin, 
        "destination": destination, 
        "guest_count": guest_count, 
        "genre": genre,
        "date": date,
        "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
        )


In [47]:
from langchain.agents import create_agent

coordinator = create_agent(
    model=model,
    tools=[search_flights, search_venues, suggest_playlist, update_state],
    state_schema=WeddingState,
    system_prompt="""
    You are a wedding coordinator. Delegate tasks to your specialists for flights, venues and playlists.
    First find all the information you need to update the state. Once that is done you can delegate the tasks.
    Once you have received their answers, coordinate the perfect wedding for me.
    """
)


## Test


In [48]:
from langchain.messages import HumanMessage

response = await coordinator.ainvoke(
    {
        "messages": [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre. Date is 26/05/2026")],
    }
)

In [49]:
from pprint import pprint

pprint(response)

{'date': '26/05/2026',
 'destination': 'Paris',
 'genre': 'jazz',
 'guest_count': '100',
 'messages': [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre. Date is 26/05/2026", additional_kwargs={}, response_metadata={}, id='0fee6717-2b87-4d9d-9d41-f4eab07a1908'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 226, 'total_tokens': 264, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-Cr6crgTcyQPh159qG63yKFdD8OdQx', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'de

In [51]:
print(response["messages"][-1].content)

Here's the coordinated plan for your wedding in Paris on May 26, 2026:

### Flights
- **Recommended Flight**:  
  - **London Gatwick (LGW) to Paris Charles de Gaulle (CDG):**  
    Departure: 5:05 PM, Arrival: 7:20 PM (1h 15m)  
    **Price**: $101 (Economy)  
    [Book Here](https://on.kiwi.com/2x91Pj)  

This flight offers the best balance between price and timing for travel from London to Paris.

---

### Venue Options
1. **Hotel Alfred Sommier**  
   - Capacity: Up to 180 guests.  
   - Features: Six reception salons, rooftop terrace, private garden (100m²).  
   - Notes: Perfect for a grand, elegant wedding.  
   
2. **Domaine d'Aveny** (*Recommended*)  
   - Capacity: Ideal for 100 guests, accommodation for 94 across 34 en-suite rooms.  
   - Notes: Enchanting castle ambiance, highly rated, exclusive hire starts at €11,500.  

3. **Paris Marriott Opera Ambassador Hotel**  
   - Spaces: 9 event rooms.  
   - Notes: Convenient city location, professional event hosting.  

**Recomme

link to trace: https://smith.langchain.com/public/7b5fe668-d3e3-4af4-b513-a8cacc0c9e84/r